In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from tqdm import tqdm

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

In [2]:
root_dir = '../'

cfg.init(filename=f"{root_dir}/config.yaml")

input_dir = f"{cfg.config['out_dir']}/out-publish-normalized"

In [3]:
COMBINE_harmonizer.init_redcap(token=cfg.config["redcap_token"], host=cfg.config["redcap_host"])

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'
_REDCAP_MAIN_MAP = COMBINE_harmonizer.build_redcap_column_map(data_dict_filename, COMBINE_harmonizer.SHEET_MAIN)

In [5]:
filename_infos: list[COMBINE_harmonizer.REDCapFilenameInfo] = [
    {
        'name': '01-02-screening.csv',
        'data_dict': COMBINE_harmonizer.SHEET_MAIN,
        'form_name': 'identifier',
        'map': _REDCAP_MAIN_MAP,
        'id': '01_02',
        'category': COMBINE_harmonizer.CATEGORY_PRE_INTERVENTION,
        'subcategory': COMBINE_harmonizer.SUBCATEGORY_IDENTITY,
    },
]

## 01-Main

In [6]:
filename_info = filename_infos[0]

columns = ['_study', 'center', 'subjectID', 'uniqueID']
column_name_map = {
    '_study': 'study',
    'center': 'center',
    'subjectID': 'subject_id',
    'uniqueID': 'unique_id',
}
df_records = COMBINE_harmonizer.redcap_normalize_filename_info(filename_info, input_dir, valid_columns=columns, column_name_map=column_name_map, exclude_columns=[])

In [7]:
all_records = [dict(row) for idx, row in df_records.iterrows()]

In [8]:
for record in tqdm(all_records):
    COMBINE_harmonizer.put_redcap_data('record', [record])

100%|██████████| 532/532 [10:43<00:00,  1.21s/it]
